In [1]:
##### Takes a search query as input and get the vectors from the whole dataset to compare.
import spacy
import pickle
import numpy as np
from scipy import spatial
import sys
import unidecode
#from sklearn.decomposition import PCA
#QUERY  Neighbours Ids_and_Score_bool
directory='../'
argv=sys.argv
nlp = spacy.load("fr_core_news_lg")
pca = pickle.load(open(directory+'models/pca_30.pkl','rb'))
pca_space= np.load(directory+'models/vectors_pca_30.npy', allow_pickle=True)
id_table=list(np.load(directory+'../data/id_table.npy', allow_pickle=True))
tree = spatial.KDTree(pca_space)
from spacy.lang.fr.stop_words import STOP_WORDS
from spacy.lang.fr import French
parser=French()
stopwords = list(STOP_WORDS)

def process_query(search_query):
    query=str(search_query).lower()
    clean_query = unidecode.unidecode(query)
    tokens=parser(clean_query)
    tokens = [ word.lower_ for word in tokens ]
    tokens = [ word for word in tokens if word not in stopwords]
    tokens = " ".join([i for i in tokens])
    return (tokens)

def query2vec(search_query):
    x=nlp(search_query).vector #spacy 300d
    y=pca.transform([x])[0] #pca 30d
    return(y)

def get_id(idx):
    dataset_id=id_table[idx-1]
    return(dataset_id)

def get_idx(ids):
    dataset_idx=id_table.index(ids)+1
    return(dataset_idx)

def id2vec(ids):
    return(pca_space[get_idx(ids)])

def neighbours(vector, n):
    n_ids=[]
    score=[]
    dist, pos=tree.query(vector, k=n)
    for j in range(len(pos)):
        n_ids.append(get_id(pos[j]))
        score.append(1-dist[j]/50) ##very approximate metric 
    return(n_ids, score)

def Search(search_query, n):
    n_ids, score=neighbours(query2vec(process_query(search_query)), n)
    #print(n_ids, score)
    return(n_ids, score)

def Similarity(ids, n):
    n_ids, score=neighbours(id2vec(ids), n)
    return(n_ids, score)

In [3]:
import pandas as pd
df=pd.read_csv('../../data/querys.csv', sep=',',error_bad_lines=False, encoding='latin-1') #le fichier original

In [4]:
data=pd.read_pickle("../../data/nodesc_clean.pkl")
data_np=data.to_numpy()

In [5]:
pca_space[get_idx('53ba4c07a3a729219b7bead3')]

array([-0.39525387, -1.80858754, -0.99972246, -1.79877351,  3.04200497,
       -1.07497441,  1.87230447,  0.96031802,  0.22992027,  1.68786773,
       -1.46389797,  1.89697827,  0.16721121,  1.1637145 , -1.08350189,
       -0.50725967,  0.33706491,  0.6243299 ,  0.64117906,  1.80798095,
        0.43165287, -0.00331491, -2.100112  , -0.93477975,  0.91513703,
       -0.34883433, -0.66823246, -0.02884761,  0.41350963, -0.40011595])

In [6]:
df_array=np.array(df)

In [51]:
for i in range(len(df_array)):
    n_ids, score = Similarity(df_array[i][2], 10)
    print('\n'+df_array[i][0])
    for k in range(len(n_ids)):
        print('Neighbour n°: '+ str(k) + data_np[get_idx(n_ids[k])])


siren
Neighbour n°: 0base sirene des entreprises et de leurs établissements siren siret . association companies entreprises etablissements register registre repertoire siren sirene siret
Neighbour n°: 1base sirene v3 département du gers. analytique entreprises etablissements legale siren sirene siret
Neighbour n°: 2liste des entreprises base sirene v3 commune de lorient. administration business citizenship economic development economy employment entreprises government insee lorient public finances siren sirene siret sme ville de lorient
Neighbour n°: 3base sirene des entreprises des hautes pyrénées. entreprises etablissements legale siren sirene siret territoire
Neighbour n°: 4base sirene entreprises corses. administration business citizenship economic development economy employment entreprises government insee public finances siren sirene siret sme
Neighbour n°: 5base sirene des entreprises de haute garonne. entreprises insee siren sirene siret territoire
Neighbour n°: 6base sirene d

TypeError: can only concatenate str (not "float") to str

In [8]:
scores=[]
positions=[]
for i in range(len(df_array)):
    n_ids, score = Search(df_array[i][0], 20)
    #print(n_ids)
    #print(df_array[i][2])
    for k in range(len(n_ids)):
        if n_ids[k]==df_array[i][2]:
            score=1-k/len(n_ids)
            pos=k
            print('FOUND :' + str(df_array[i][0])+ ' in pos '+ str(k))
            break
        else:
            score=0
            pos=k
            if k==len(n_ids)-1:
                print('DIDNT FIND:'+str(df_array[i][0]))
    scores.append(score)
    positions.append(k)

5b7ffc618b4c4169d30727e0
DIDNT FIND:siren
5b7ffc618b4c4169d30727e0
DIDNT FIND:sirene
5b7ffc618b4c4169d30727e0
DIDNT FIND:entreprise
5b7ffc618b4c4169d30727e0
DIDNT FIND:entreprises
5b7ffc618b4c4169d30727e0
DIDNT FIND:siret
54de1e8fc751df388646738b
DIDNT FIND:open damir
54de1e8fc751df388646738b
DIDNT FIND:opendamir
54de1e8fc751df388646738b
DIDNT FIND:damir
536991b0a3a729239d203d13
FOUND :contours départements in pos 6
53ba4c07a3a729219b7bead3
DIDNT FIND:emissions polluantes
536995f5a3a729239d20487f
DIDNT FIND:géofla départements
5369986ba3a729239d204f55
FOUND :effectifs police municipale in pos 10
5a0effeac751df5832ded865
FOUND :marchés public bourgogne in pos 2
59593619a3a7291dd09c8238
FOUND :Liste gares SNCF in pos 0
536991b0a3a729239d203d13
DIDNT FIND:contours départements français
5625fba688ee3820e912613c
FOUND :loi de finance 2016 in pos 9
5625fba688ee3820e912613c
DIDNT FIND:lolf 2016
53699058a3a729239d2039a3
DIDNT FIND:formations pas de calais
536997eaa3a729239d204dfd
DIDNT FIND:ac

In [9]:
positions

[19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 6,
 19,
 19,
 10,
 2,
 0,
 19,
 9,
 19,
 19,
 19,
 19,
 2,
 19,
 0,
 19,
 19,
 19,
 7,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 18,
 19]

In [52]:
df

,query,params,expected
0,siren,NaN,5b7ffc618b4c4169d30727e0
1,sirene,NaN,5b7ffc618b4c4169d30727e0
2,entreprise,NaN,5b7ffc618b4c4169d30727e0
3,entreprises,NaN,5b7ffc618b4c4169d30727e0
4,siret,NaN,5b7ffc618b4c4169d30727e0
5,open damir,NaN,54de1e8fc751df388646738b
6,opendamir,NaN,54de1e8fc751df388646738b
7,damir,NaN,54de1e8fc751df388646738b
8,contours départements,NaN,536991b0a3a729239d203d13
9,emissions polluantes,NaN,53ba4c07a3a729219b7bead3


In [10]:
scores

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.94,
 0,
 0,
 0.9,
 0.98,
 1.0,
 0,
 0.91,
 0,
 0,
 0.55,
 0,
 0.98,
 0,
 1.0,
 0,
 0,
 0,
 0.9299999999999999,
 0,
 0,
 0.61,
 0,
 0,
 0,
 0,
 0.8,
 0,
 0,
 0,
 0,
 0,
 0.8200000000000001,
 0]

In [48]:
df=df.drop(17)

In [17]:
get_id(40989)

'53699489a3a729239d204488'